In [ ]:
! pip install langchain_core langchain-anthropic langgraph 

In [1]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

In [2]:
# LLM
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-3-5-sonnet-latest")

### Vanilla Agent

* No orchestration framework 
* Use LangChain to bind tools and specify tools 

In [26]:
from langchain_core.tools import tool

# Define tools
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

# Augment the LLM with tools
tools = [add, multiply, divide]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langgraph.graph import add_messages
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    BaseMessage,
    ToolCall,
)

def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    return llm_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
            )
        ]
        + messages
    )

def call_tool(tool_call: ToolCall):
    """Performs the tool call"""

    tool = tools_by_name[tool_call["name"]]
    return tool.invoke(tool_call)

def agent(messages: list[BaseMessage]):
    """ Tool calling agent """
    llm_response = call_llm(messages)

    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_results = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        messages = add_messages(messages, [llm_response, *tool_results])
        llm_response = call_llm(messages)

    messages = add_messages(messages, llm_response)
    return messages

# Stream
messages = agent([HumanMessage(content="Add 3 and 4.")])
for m in messages:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

[{'text': "I'll help you add 3 and 4 using the `add` function.", 'type': 'text'}, {'id': 'toolu_01Js4XXxB1oMDqZKz6VkxpnQ', 'input': {'a': 3, 'b': 4}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_01Js4XXxB1oMDqZKz6VkxpnQ)
 Call ID: toolu_01Js4XXxB1oMDqZKz6VkxpnQ
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


### Agent with short-term memory (within a thread)

* LangGraph persistence layer
* Allows you to resume any conversation with the agent

`@entrypoint` 
* Decorator indicates the start of a workflow / agent 
* Produces a Pregel object, an abstraction for managing a few things
* Execution -- Syncronous (invoke), Async (ainvoke), streaming (stream)
* State -- Checkpointing, Human in the loop (interrupt)

[Optional: `@entrypoint.final`](https://langchain-ai.github.io/langgraph/concepts/functional_api/#entrypointfinal)
* Can be used to specify what to return vs what to checkpoint 

`@task`
* Results from tasks are saved as checkpoints
* Important for caching results (time-consuming operations)
* Support streaming updates from tasks
* Support tracing

Calling a task -- 
* When you call a task, it returns immediately with a future object.
* A future is a placeholder for a result that will be available later.
* `.result()` marks where in the code you actually need the task's result.

In [5]:
import uuid
from langgraph.func import entrypoint, task # New 
from langgraph.checkpoint.memory import MemorySaver # New 

@task # New
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    return llm_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
            )
        ]
        + messages
    )

@task # New
def call_tool(tool_call: ToolCall):
    """Performs the tool call"""

    tool = tools_by_name[tool_call["name"]]
    return tool.invoke(tool_call)

checkpointer = MemorySaver() # New
@entrypoint(checkpointer=checkpointer) # New 
def agent(messages: list[BaseMessage], previous: list[BaseMessage]): # previous (state associated with the previous ckpt)
    """ Tool calling agent """

    # Add previous messages from short-term memory to the current messages
    if previous is not None:
        messages = add_messages(previous, messages)
    
    # Call the LLM
    llm_response = call_llm(messages).result()

    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_results = [
            call_tool(tool_call).result() for tool_call in llm_response.tool_calls
        ]
        messages = add_messages(messages, [llm_response, *tool_results])
        llm_response = call_llm(messages).result()

    messages = add_messages(messages, llm_response)

    # Return LLM response and save the full message history
    return messages

In [6]:
# Thread ID
thread_id = str(uuid.uuid4())

# Config
config = {"configurable": {"thread_id": thread_id}}

# Run with checkpointer to persist state in memory
messages = agent.invoke([HumanMessage(content="Add 3 and 4.")], config)
for m in messages:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

[{'text': "I'll help you add 3 and 4 using the `add` function.", 'type': 'text'}, {'id': 'toolu_016nz34fiRCrz2pcipHGRbEc', 'input': {'a': 3, 'b': 4}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_016nz34fiRCrz2pcipHGRbEc)
 Call ID: toolu_016nz34fiRCrz2pcipHGRbEc
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [7]:
# Get the last checkpoint, which contains the full message history
agent_state = agent.get_state(config)
for m in agent_state.values:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

[{'text': "I'll help you add 3 and 4 using the `add` function.", 'type': 'text'}, {'id': 'toolu_016nz34fiRCrz2pcipHGRbEc', 'input': {'a': 3, 'b': 4}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_016nz34fiRCrz2pcipHGRbEc)
 Call ID: toolu_016nz34fiRCrz2pcipHGRbEc
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [8]:
# Continue with the same thread
messages = agent.invoke([HumanMessage(content="Take the result and multiply it by 2.")], config)
for m in messages:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

[{'text': "I'll help you add 3 and 4 using the `add` function.", 'type': 'text'}, {'id': 'toolu_016nz34fiRCrz2pcipHGRbEc', 'input': {'a': 3, 'b': 4}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_016nz34fiRCrz2pcipHGRbEc)
 Call ID: toolu_016nz34fiRCrz2pcipHGRbEc
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.
================================ Human Message =================================

Take the result and multiply it by 2.
================================== Ai Message ==================================

[{'text': "I'll take the previous result (7) and multiply it by 2 using the `multiply` function.", 'type': 'text'}, 

### Agent with HITL

* Very useful for [approval](https://www.anthropic.com/research/building-effective-agents) in agents.
* Add interrupt to the workflow to allow for HITL.
* Re-executes from the start of the entrypoint.
* Output of each `@task` is cached / saved as a checkpoint.

In [37]:
from langgraph.types import interrupt

@task
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    print("Calling LLM!")
    return llm_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
            )
        ]
        + messages
    )

@task
def call_tool(tool_call: ToolCall):
    """Performs the tool call"""
    print("Calling tool!")
    # Interrupt the workflow to get a review from a human.
    is_approved = interrupt({ # New 
            # Any json-serializable payload provided to interrupt as argument.
            # It will be surfaced on the client side as an Interrupt when streaming data
            # from the workflow.
            "tool_call": tool_call, # The tool call we want reviewed.
            # We can add any additional information that we need.
            # For example, introduce a key called "action" with some instructions.
            "action": "Please approve/reject the tool call",
        })
    
    if is_approved:
        tool = tools_by_name[tool_call["name"]]
        return tool.invoke(tool_call)
    else:
        return "Tool call rejected"

@entrypoint(checkpointer=MemorySaver())  
def agent(messages: list[BaseMessage], previous: list[BaseMessage]): 
    """ Tool calling agent """
    print("Executing agent!")
    
    # Add previous messages from short-term memory to the current messages
    if previous is not None:
        messages = add_messages(previous, messages)
    
    # Call the LLM
    llm_response = call_llm(messages).result()

    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_results = [
            call_tool(tool_call).result() for tool_call in llm_response.tool_calls
        ]
        messages = add_messages(messages, [llm_response, *tool_results])
        llm_response = call_llm(messages).result()

    messages = add_messages(messages, llm_response)
    return messages

In [38]:
# Thread ID
thread_id = str(uuid.uuid4())

# Config
config = {"configurable": {"thread_id": thread_id}}

# Run until the interrupt 
for item in agent.stream([HumanMessage(content="Add 3 and 4.")], config, stream_mode="updates"):
    if '__interrupt__' in item:
        print(item['__interrupt__'][0].value)

Executing agent!
Calling LLM!
Calling tool!
{'tool_call': {'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'toolu_01GGBdQZJ11bZGyHuvRkuRrh', 'type': 'tool_call'}, 'action': 'Please approve/reject the tool call'}


In [39]:
from langgraph.types import Command
for item in agent.stream(Command(resume=True), config, stream_mode="updates"):
    if 'agent' in item:
        item['agent'][-1].pretty_print()

Executing agent!
Calling tool!
Calling LLM!
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


Initial Execution Flow:
* `call_llm` runs first and is cached
* `call_tool` starts but hits the interrupt
* Execution pauses waiting for human input

After Resume:
* `call_tool` continues from where it left off (since it didn't complete)
* The tool executes if approved
* `call_llm` runs again with the tool results
* Final response is generated

### Time travel

* It can be useful to add time travel to the workflow.
* This allows you to rewind to specific checkpoint, modify the workflow, and re-run it. 


In [11]:
@task
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    return llm_with_tools.invoke(
        [
            SystemMessage(
                content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
            )
        ]
        + messages
    )

@task
def call_tool(tool_call: ToolCall):
    """Performs the tool call"""
    tool = tools_by_name[tool_call["name"]]
    return tool.invoke(tool_call)

checkpointer = MemorySaver()
@entrypoint(checkpointer=checkpointer) 
def agent(messages: list[BaseMessage], previous: list[BaseMessage]): # New 
    """ Tool calling agent """
    # Add previous messages from short-term memory to the current messages
    if previous is not None:
        messages = add_messages(previous, messages)
    
    # Call the LLM
    llm_response = call_llm(messages).result()

    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_results = [
            call_tool(tool_call).result() for tool_call in llm_response.tool_calls
        ]
        messages = add_messages(messages, [llm_response, *tool_results])
        llm_response = call_llm(messages).result()

    messages = add_messages(messages, llm_response)
    return messages

In [12]:
# Thread ID
thread_id = str(uuid.uuid4())

# Config
config = {"configurable": {"thread_id": thread_id}}

# Run with checkpointer to persist state in memory
messages = agent.invoke([HumanMessage(content="Add 3 and 4.")], config)
for m in messages:
    m.pretty_print()


================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

[{'text': "I'll help you add 3 and 4 using the `add` function.", 'type': 'text'}, {'id': 'toolu_0144yacRC2U5UkwEfEzBdzm2', 'input': {'a': 3, 'b': 4}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_0144yacRC2U5UkwEfEzBdzm2)
 Call ID: toolu_0144yacRC2U5UkwEfEzBdzm2
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [13]:
# Second turn
for item in agent.stream([HumanMessage(content="Take the result and multiply it by 2.")], config, stream_mode="updates"):
    if 'agent' in item:
        item['agent'][-1].pretty_print()

================================== Ai Message ==================================

7 multiplied by 2 equals 14.


In [14]:
# Fork and do alternative second turn
to_fork_from = list(agent.get_state_history(config))[1].config
to_fork_from

{'configurable': {'thread_id': 'c0336c2c-1a97-4b6d-826c-21c33eb47ccb',
  'checkpoint_ns': '',
  'checkpoint_id': '1efddbc1-103d-66d4-8001-48133c20ef14'}}

In [15]:
# Get the last checkpoint, which contains the full message history
agent_state = agent.get_state(to_fork_from)
for m in agent_state.values:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4.
================================== Ai Message ==================================

[{'text': "I'll help you add 3 and 4 using the `add` function.", 'type': 'text'}, {'id': 'toolu_0144yacRC2U5UkwEfEzBdzm2', 'input': {'a': 3, 'b': 4}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_0144yacRC2U5UkwEfEzBdzm2)
 Call ID: toolu_0144yacRC2U5UkwEfEzBdzm2
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: add

7
================================== Ai Message ==================================

The sum of 3 and 4 is 7.


In [16]:
# Re-run the workflow from the fork
for item in agent.stream([HumanMessage(content="Take the result and multiply it by 3.")], to_fork_from, stream_mode="updates"):
    if 'agent' in item:
        item['agent'][-1].pretty_print()

================================== Ai Message ==================================

7 multiplied by 3 equals 21.


### Agent with HITL and Long-term memory (across threads)

* Add interrupt to the workflow to allow for HITL
* Add tool for [long-term memory](https://langchain-ai.github.io/langgraph/concepts/memory/#long-term-memory)

In [48]:
import uuid
from typing import Annotated, Optional

from langchain_core.tools import InjectedToolArg
from langgraph.store.base import BaseStore

@tool 
def upsert_memory(
    content: str,
    *,
    memory_id: Optional[uuid.UUID] = None,
    # Hide these arguments from the model.
    store: Annotated[BaseStore, InjectedToolArg],
):
    """Upsert a memory in the database.

    If a memory conflicts with an existing one, then just UPDATE the
    existing one by passing in memory_id - don't create two memories
    that are the same. If the user corrects a memory, UPDATE it.

    Args:
        content: The main content of the memory. For example:
            "User expressed interest in learning about French."
        memory_id: ONLY PROVIDE IF UPDATING AN EXISTING MEMORY.
        The memory to overwrite.
    """
    mem_id = memory_id or uuid.uuid4()

    # BaseStore is a LangGraph persistence layer
    store.put(
        ("memories","lance"),
        key=str(mem_id),
        value={"content": content},
    )
    return f"Stored memory {mem_id}"

# Augment the LLM with tools
tools = [upsert_memory]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_memory_tool = llm.bind_tools(tools)

In [50]:
from langgraph.store.memory import InMemoryStore # New 
from langchain_core.messages import ToolMessage

@task
def call_llm(messages: list[BaseMessage]):
    """LLM decides whether to call a tool or not"""
    return llm_with_memory_tool.invoke( # New 
        [
            SystemMessage(
                content="You are a helpful assistant tasked with storing memories." # New 
            )
        ]
        + messages
    )

@task
def call_tool(tool_call: ToolCall, store: BaseStore):

    # Interrupt the workflow to get a review from a human.
    is_approved = interrupt({ # New 
            # Any json-serializable payload provided to interrupt as argument.
            # It will be surfaced on the client side as an Interrupt when streaming data
            # from the workflow.
            "tool_call": tool_call, # The tool call we want reviewed.
            # We can add any additional information that we need.
            # For example, introduce a key called "action" with some instructions.
            "action": "Please approve/reject the tool call",
        })
    
    if is_approved:

        print("Tool call approved, Memory Added!")

        tool = tools_by_name[tool_call["name"]]
        tool.invoke({**tool_call["args"], "store": store})

        # Tool message provides confirmation to the model that the actions it took were completed
        results = ToolMessage(content=tool_call["args"]["content"], tool_call_id=tool_call["id"])
        return results
    else: 
        return "Tool call rejected"

in_memory_store = InMemoryStore()
@entrypoint(checkpointer=MemorySaver(), store=in_memory_store)  
def agent(messages: list[BaseMessage], previous: list[BaseMessage], store: BaseStore): 
    """ Tool calling agent """

    # Add previous messages from short-term memory to the current messages
    if previous is not None:
        messages = add_messages(previous, messages)
    
    # New 
    # Retrieve the most recent memories for context
    memories = store.search( 
        ("memories","lance"),
        limit=10,
    )

    # New
    # Format memories for inclusion in the prompt
    formatted = "\n".join(f"[{mem.key}]: {mem.value} (similarity: {mem.score})" for mem in memories)
    if formatted:
        formatted = f"""
<memories>
{formatted}
</memories>"""

    # New
    # Call the LLM
    llm_response = call_llm([SystemMessage(content=f"Here is some context for you about the user: {formatted}"), *messages]).result()

    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_results = [
            call_tool(tool_call=tool_call, store=store).result() for tool_call in llm_response.tool_calls
        ]
        messages = add_messages(messages, [llm_response, *tool_results])
        llm_response = call_llm(messages).result()

    messages = add_messages(messages, llm_response)
    return messages

In [51]:
# Thread ID
thread_id = str(uuid.uuid4())

# Config
config = {"configurable": {"thread_id": thread_id}}

# Run until the interrupt 
for item in agent.stream([HumanMessage(content="Hi my name is Lance.")], config, stream_mode="updates"):
    if '__interrupt__' in item:
        print(item['__interrupt__'][0].value)

{'tool_call': {'name': 'upsert_memory', 'args': {'content': "User's name is Lance."}, 'id': 'toolu_01Y1J6BnLWejpoQfF72GWP9L', 'type': 'tool_call'}, 'action': 'Please approve/reject the tool call'}


In [52]:
for item in agent.stream(Command(resume=True), config, stream_mode="updates"):
    if 'agent' in item:
        item['agent'][-1].pretty_print()

Tool call approved, Memory Added!
================================== Ai Message ==================================

Nice to meet you Lance! Is there anything else you'd like me to help you with?


In [55]:
in_memory_store.search(("memories","lance"))

[Item(namespace=['memories', 'lance'], key='20544090-a509-46ac-85f2-04b3f54db0d2', value={'content': "User's name is Lance."}, created_at='2025-01-28T21:41:25.959474+00:00', updated_at='2025-01-28T21:41:25.959475+00:00', score=None)]

### Overview

- Vanilla agent
- Added short-term memory (w/ tracing)
- Added short-term memory + HITL (w/ tracing)
- Added short-term memory + HITL + time travel (w/ tracing)
- Added short-term memory + HITL + long-term memory (w/ tracing)